In [1]:
import os
import torch
import pickle

from torchinfo import summary

In [2]:
from models import (
    make_resnet_pretrained_classifier,
    make_slowfast_pretrained_classifier,
    make_slow_pretrained_segmenter)
from lared_laughter.constants import models_path

In [3]:
pretrained = make_slow_pretrained_segmenter()
# pretrained = make_slowfast_pretrained_classifier()

In [ ]:
input = [
    torch.rand((8, 3, 8, 256, 256)), # slow
    torch.rand((8, 3, 32, 256, 256)) # fast
]

In [4]:
# summary(pretrained, input_data=input, device='cuda', depth=5)
summary(pretrained, input_size=(8, 3, 8, 256, 256), device='cuda', depth=5)

/home/jose/miniconda3/lib/python3.8/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


Layer (type:depth-idx)                                  Output Shape              Param #
PTVResNet                                               [8, 60]                   --
├─Net: 1-2                                              [8, 2048, 8, 8, 8]        (recursive)
│    └─ModuleList: 2-1                                  --                        9,536
│    │    └─ResNetBasicStem: 3-3                        [8, 64, 8, 64, 64]        (recursive)
│    │    │    └─Conv3d: 4-5                            [8, 64, 8, 128, 128]      (recursive)
│    │    │    └─BatchNorm3d: 4-6                       [8, 64, 8, 128, 128]      (recursive)
│    │    │    └─ReLU: 4-7                              [8, 64, 8, 128, 128]      --
│    │    │    └─MaxPool3d: 4-8                         [8, 64, 8, 64, 64]        --
│    │    └─ResStage: 3-4                               [8, 256, 8, 64, 64]       (recursive)
├─Net: 1-2                                              [8, 2048, 8, 8, 8]        (recursive)
│  

In [5]:
pretrained.model.blocks

ModuleList(
  (0): ResNetBasicStem(
    (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
    (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU(inplace=True)
    (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
  )
  (1): ResStage(
    (res_blocks): ModuleList(
      (0): ResBlock(
        (branch1_conv): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (branch1_norm): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (branch2): BottleneckBlock(
          (conv_a): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (norm_a): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_a): ReLU(inplace=True)
          (conv_b): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1),

In [6]:
pretrained.head

SegmentationHead(
  (head): Sequential(
    (0): AvgPool3d(kernel_size=(1, 8, 8), stride=(1, 1, 1), padding=(0, 0, 0))
    (1): Conv3d(2048, 1, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
  )
  (upsample): Sequential(
    (0): Flatten(start_dim=2, end_dim=-1)
    (1): Upsample(size=90, mode=linear)
  )
)